In [43]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import psycopg2
import pickle

In [90]:
ubuntu=True

con = None

if ubuntu:
    con = psycopg2.connect(database='mountainproject', password='kepler31', user='devin', port=5432, host='/var/run/postgresql/')
else:
    con = psycopg2.connect(database='mountainproject', user='User')
cur = con.cursor()
con.autocommit = True

In [49]:

def get_user_id_gte(table='stars', n=5):
    #user_ids from users greater than or equal to n reviews
    user_id_gte= '''
        SELECT user_id
        FROM {}
        GROUP BY user_id
        HAVING COUNT(user_id) >= %s;
        '''.format(table)
    cur.execute(user_id_gte, (n,) )
    tup_list = cur.fetchall()
    #return tup_list
    return [tup[0] for tup in tup_list]
#user_id_list = get_user_id_gte('stars', 5), len(user_id_list)


In [50]:
def get_climb_id_from_user_id_gte(id_tuple, n, table='stars'):
    climb_ids = '''
        SELECT climb_id
        FROM {}
        WHERE user_id IN %s
        GROUP BY climb_id
        HAVING COUNT(climb_id) >= %s;
    '''.format(table)
    cur.execute(climb_ids, (id_tuple, n) )
    return [tup[0] for tup in cur.fetchall()]
#climb_id_list = get_climb_id_from_user_id_gte(tuple(user_id_list), 5), len(climb_id_list)


In [51]:
def get_user_id_from_climb_id_gte(id_tuple, n, table='stars'):
    #Get user ids with at least n reviews in the climb id tuple
    user_ids_sql = '''
        SELECT user_id
        FROM {}
        WHERE climb_id IN %s
        GROUP BY user_id
        HAVING COUNT(user_id) >= %s;
    '''.format(table)
    cur.execute(user_ids_sql, (id_tuple, n) )
    return [tup[0] for tup in cur.fetchall()]
#user_id_list2 = get_user_id_from_climb_id_gte(tuple(climb_id_list), 5), len(user_id_list2)


In [94]:
def find_climb_user_ids(table='stars', n=5):
    climb_id_old = [0]
    climb_id_new = [-2]
    user_id_old = [0]
    user_id_new = [-2] 
    user_id_new = get_user_id_gte(table=table, n=n)
    while (set(user_id_old) != set(user_id_new)) and \
            (set(climb_id_old) != set(climb_id_new)):
        print "climb list:", len(climb_id_new)
        print "user list:", len(user_id_new)
        climb_id_old = climb_id_new
        climb_id_new = get_climb_id_from_user_id_gte(tuple(user_id_new), 
                                                     n,
                                                     table=table)
        user_id_old = user_id_new
        user_id_new = get_user_id_from_climb_id_gte(tuple(climb_id_new),
                                                    n,
                                                    table=table)
        
    return user_id_new, climb_id_new

#(user_id_list, climb_id_list) = find_climb_user_ids(table='stars', n=5)

In [95]:
def pickle_id_list(id_list, name):
    # Save id list for future use
    with open('id_list_'+name+'.p','w') as f:
        pickle.dump(id_list, f)

In [97]:
(user_id_list, climb_id_list) = find_climb_user_ids(n=5)

climb list: 1
user list: 7592
climb list: 13694
user list: 7565
climb list: 13678
user list: 7562
climb list: 13676
user list: 7561


In [98]:
pickle_id_list(climb_id_list, 'climb_star5')
pickle_id_list(user_id_list, 'user_star5')

In [55]:
(user_id_list_sub, climb_id_list_sub) = find_climb_user_ids(n=30)

climb list: 1
user list: 2558
climb list: 2508
user list: 1555
climb list: 2042
user list: 1415
climb list: 1931
user list: 1367
climb list: 1897
user list: 1352
climb list: 1884
user list: 1350
climb list: 1879
user list: 1344
climb list: 1871
user list: 1337
climb list: 1864
user list: 1331
climb list: 1860
user list: 1329
climb list: 1857
user list: 1327
climb list: 1854
user list: 1326


In [56]:
pickle_id_list(climb_id_list_sub, 'climb_star30')
pickle_id_list(user_id_list_sub, 'user_star30')

In [234]:
(user_id_list_sub, climb_id_list_sub) = find_climb_user_ids(n=35)

climb list: 1
user list: 2255
climb list: 1940
user list: 1190
climb list: 1423
user list: 948
climb list: 1201
user list: 839
climb list: 1082
user list: 773
climb list: 1016
user list: 740
climb list: 983
user list: 724
climb list: 973
user list: 718
climb list: 964
user list: 716
climb list: 959
user list: 712
climb list: 951
user list: 706
climb list: 944
user list: 702
climb list: 942
user list: 701
climb list: 940
user list: 700


In [235]:
pickle_id_list(climb_id_list_sub, 'climb_star35')
pickle_id_list(user_id_list_sub, 'user_star35')

###Generate Sparse Matrix for Analysis

In [92]:
from scipy import sparse as sp

In [130]:
with open('id_list_'+'climb_star35'+'.p','r') as f:
        climb_ids = pickle.load(f)
len(climb_ids)

940

In [131]:
with open('id_list_'+'user_star35'+'.p','r') as f:
        user_ids = pickle.load(f)
len(user_ids)

700

In [133]:
user_set = set(user_ids)
climb_set = set(climb_ids)
climb_tuple = tuple(climb_ids)

climb_dict = {}
for i, el in enumerate(climb_tuple):
    climb_dict[el]=i

data = []
row = []
col = []
table = 'stars'
for i, user in enumerate(user_ids):
    #print user
    q = '''
    SELECT user_stars, climb_id
    FROM {}
    WHERE user_id = %s
    AND climb_id IN %s
    '''.format(table)
    cur.execute(q, (user, climb_tuple) )
    for el in cur.fetchall():
        data += [el[0]]
        row += [i]
        col += [climb_dict[el[1]]]
    
sp_mat_input = (data,(row,col))

In [137]:
sp_mat = sp.coo_matrix(sp_mat_input, shape=(len(user_ids), len(climb_ids)) )

In [147]:
sp_mat.todense()

matrix([[ 4, 14,  0, ...,  0,  4,  4],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  4, ...,  0,  0,  0],
        ..., 
        [ 0,  0,  0, ...,  0,  4,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0]])

In [139]:
sp_mat.shape

(700, 940)

##Matrix Factorization


In [59]:
import numpy as np

In [164]:
def matrix_factorization(R, K=10, steps=5000, alpha=0.0002, beta=0.02):
    (N, M) = R.shape
    P = numpy.random.rand(N,K)
    Q = numpy.random.rand(K,M)
    
    for step in xrange(steps):
        
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i,j]>0:
                    eij = R[i,j] - np.dot(P[i,:], Q[:,j])
                    for k in xrange(K):
                        P[i,k]=P[i,k]+alpha*(2*eij*Q[k,j]-beta*P[i,k])
                        Q[k,j]=Q[k,j]+alpha*(2*eij*P[i,k]-beta*Q[k,j])
        eR = np.dot(P,Q)
        e = 0
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i,j] > 0:
                    e=e+pow(R[i,j]-np.dot(P[i,:],Q[:,j]),2)
                    for k in xrange(K):
                        e=e+(beta/2)*(pow(P[i,k],2)+pow(Q[k,j],2))
        if e < 0.001:
            print 'Reached error limit!'
            break
    return P, Q, e, step

In [237]:
nP, nQ, e, step = matrix_factorization(sp_mat.todense(), steps = 500000)

In [238]:
np.matrix(nP.dot(nQ))

matrix([[ 3.98709657,  1.91706835,  3.39486241, ...,  3.56383391,
          3.56032769,  3.84027654],
        [ 3.61863751,  1.63152471,  3.11873247, ...,  3.63052796,
          3.88611762,  3.68313947],
        [ 2.80502908,  1.25474518,  2.29519135, ...,  2.82911618,
          2.79975728,  3.30629069],
        ..., 
        [ 2.25831332,  1.20152505,  1.96232778, ...,  2.12020517,
          1.56147965,  1.71387607],
        [ 4.0050998 ,  2.09087924,  3.69008406, ...,  4.09630719,
          3.93446738,  3.80166548],
        [ 2.74751294,  1.02453587,  1.84113388, ...,  2.52659444,
          2.67400275,  2.60836738]])

2.233549169020749

In [239]:
pred = nP.dot(nQ)
y = []
yp = []

for i in range(len(sp_mat_input[0])):    
    y += [sp_mat_input[0][i]]
    yp += [ pred[sp_mat_input[1][0][i], sp_mat_input[1][1][i]] ]
zip(y,yp)

[(3, 4.5444993500018027),
 (3, 3.2517888168038906),
 (5, 2.7348700076470012),
 (4, 4.2071171933975631),
 (5, 4.1511442488684089),
 (4, 2.2313725720684978),
 (5, 3.0812855345418795),
 (4, 3.3284428939071864),
 (5, 3.0777410565668273),
 (4, 1.8456099879196866),
 (3, 3.8215453625142608),
 (4, 4.8268478015225229),
 (4, 3.5418508677273994),
 (5, 3.9721414429932755),
 (4, 3.1319962453025729),
 (4, 4.424506420682472),
 (5, 3.6834340449547698),
 (5, 3.6834340449547698),
 (4, 3.3794730750071267),
 (3, 3.6980529734141112),
 (3, 3.6305365253741622),
 (4, 2.2618555630678507),
 (5, 2.9864744719502978),
 (5, 2.9864744719502978),
 (5, 3.6932226162269997),
 (4, 4.5904797763155516),
 (5, 3.8664202366229889),
 (5, 4.0693283578684891),
 (4, 4.0693283578684891),
 (4, 4.8560673230636677),
 (4, 3.0158229781539889),
 (5, 3.4257261753184141),
 (4, 3.9342082555786688),
 (3, 3.5997635178529799),
 (4, 3.2149737015882511),
 (5, 2.9943091849209118),
 (5, 3.5846843840687597),
 (4, 3.7795154989723843),
 (5, 3.409976

In [244]:
    with open('monday_ubuntu_P', 'w') as f:
        pickle.dump(nP, f)
    with open('monday_ubuntu_Q', 'w') as f:
        pickle.dump(nQ, f)

In [240]:
sp_mat.todense()

matrix([[ 4, 14,  0, ...,  0,  4,  4],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  4, ...,  0,  0,  0],
        ..., 
        [ 0,  0,  0, ...,  0,  4,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0]])

In [162]:
R = np.array([
         [5,3,0,1],
         [4,0,0,1],
         [1,1,0,5],
         [1,0,0,4],
         [0,1,5,4],
        ])
P_test, Q_test, e_test, step_test = matrix_factorization(R)
P_test.dot(Q_test)

array([[ 4.97676989,  2.98260013,  1.79322614,  1.00093821],
       [ 3.9804148 ,  2.40212123,  1.67640141,  0.99972872],
       [ 1.00018205,  0.99844221,  6.16752862,  4.97718552],
       [ 0.99881679,  0.91475177,  4.96098069,  3.98500234],
       [ 1.1418801 ,  0.99873269,  4.98695104,  3.99273906]])

In [169]:
e_test

1.1032742392919674

In [188]:
from sklearn.decomposition import NMF, ProjectedGradientNMF

In [227]:
nmfp = ProjectedGradientNMF(n_components=2).fit(R)
nmf = NMF(n_components=4, beta = 0.2, max_iter=5000, tol=1e-6,
         sparseness='data', eta = 1).fit(R)

In [228]:
nmf.components_

array([[ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 1.56130229,  0.61437869,  0.        ,  0.33381382],
       [ 0.        ,  0.25347325,  1.26736627,  1.01389301],
       [ 0.35451863,  0.19508733,  0.        ,  1.61081744]])

In [229]:
nmfp.components_

array([[ 0.        ,  0.33813856,  1.47678869,  2.71434058],
       [ 2.0925745 ,  0.79355437,  0.        ,  0.57922163]])

In [230]:
nmf.transform(R)

array([[  0.00000000e+00,   3.40561769e+00,   3.38433418e-02,
          0.00000000e+00],
       [  0.00000000e+00,   2.19188267e+00,   0.00000000e+00,
          1.35706734e-01],
       [  0.00000000e+00,   3.31989462e-02,   4.95678873e-02,
          3.07419144e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          2.46432812e+00],
       [  0.00000000e+00,   5.10309270e-16,   3.94518943e+00,
          0.00000000e+00]])

In [231]:
nmf.transform(R).dot(nmf.components_)

array([[  5.31719871e+00,   2.10091732e+00,   4.28919098e-02,
          1.17115578e+00],
       [  3.47030201e+00,   1.37312067e+00,   0.00000000e+00,
          9.50279505e-01],
       [  1.14169173e+00,   6.32696662e-01,   6.28206683e-02,
          5.01330000e+00],
       [  8.73650229e-01,   4.80759196e-01,   0.00000000e+00,
          3.96958272e+00],
       [  7.96747034e-16,   1.00000000e+00,   5.00000000e+00,
          4.00000000e+00]])

In [232]:
R

array([[5, 3, 0, 1],
       [4, 0, 0, 1],
       [1, 1, 0, 5],
       [1, 0, 0, 4],
       [0, 1, 5, 4]])

In [226]:
nmfp.transform(R).dot(nmfp.components_)

array([[ 5.25591767,  1.99316987,  0.        ,  1.4548305 ],
       [ 3.50434247,  1.32893061,  0.        ,  0.96999699],
       [ 1.31280991,  0.94419678,  1.94938256,  3.94635297],
       [ 0.98118585,  0.72182695,  1.52744717,  3.0790418 ],
       [ 0.        ,  0.65016039,  2.83951498,  5.21903424]])